In [1]:
import numpy as np
from src.visualization.vis_utils import plot_point_cloud_3d, visualize_predictions
from src.cls.prediction import create_dataloader, predict_phases
import random
from hydra import compose, initialize
from omegaconf import DictConfig, OmegaConf
from warnings import filterwarnings
filterwarnings("ignore")


In [2]:
with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name="Al_classification.yaml")
print(OmegaConf.to_yaml(cfg))

training:
  learning_rate: 0.003
  batch_size: 2096
  epochs: 100
  decay_rate: 0.002
  gpu: true
  log_every_n_steps: 5
  num_workers: 4
data:
  cube_size: 8.0
  radius: 8.0
  sample_type: regular
  sample_shape: spheric
  n_samples: 800000
  num_points: 32
  overlap_fraction: 0.3
  data_path: datasets/Al/inherent_configurations_off
  liquid_files:
  - 166ps.off
  cristal_files:
  - 240ps.off
project_name: pointnet
experiment_name: 32_points_cls_defalut_loss
experiment_tags:
- cls



In [13]:
#change config if needed
# cfg.data.sample_shape = 'spheric'
cfg.data.num_points = 64
cfg.data.radius = 7.9
cfg.data.overlap_fraction = 0.5

file_path = 'datasets/Al/inherent_configurations_off/166ps.off'
dataloader1 = create_dataloader(cfg, file_path)
file_path = 'datasets/Al/inherent_configurations_off/240ps.off'
dataloader2 = create_dataloader(cfg, file_path)

Read 1048576 points
Size of space: [267.0368   267.035942 267.03684 ]
Min coords: [-8.0e-04  5.8e-05 -8.4e-04]
Max coords: [267.036 267.036 267.036]
Avg added 0.0 points, avg dropped 49.42 points
Number of samples in spheric dataset: 32768
Read 1048576 points
Size of space: [263.22167  263.219158 263.220081]
Min coords: [ 3.30e-04  8.42e-04 -8.10e-05]
Max coords: [263.222 263.22  263.22 ]
Avg added 0.0 points, avg dropped 51.78 points
Number of samples in spheric dataset: 32768


In [14]:

points_batch_l1, coords_batch_l1 = random.choice(list(dataloader1))
points_l1 = points_batch_l1[0].numpy()
coords_l1 = np.array(coords_batch_l1)[:, 0]
print(f"Points shape: {points_l1.shape}")
print(f"Center coodinates: {coords_l1}")

fig = plot_point_cloud_3d(points_l1, n_connections=4)
fig.show()

Points shape: (64, 3)
Center coodinates: [173.7992   134.300058  15.79916 ]


In [15]:
points_batch_c1, coords_batch_c1 = random.choice(list(dataloader2))
points_c1 = points_batch_c1[0].numpy()
coords_c1 = np.array(coords_batch_c1)[:, 0]
print(f"Points shape: {points_c1.shape}")
print(f"Center coodinates: {coords_c1}")

fig = plot_point_cloud_3d(points_c1, n_connections=4)
fig.show()

Points shape: (64, 3)
Center coodinates: [ 63.20033   47.400842 142.199919]


In [20]:
from pytorch3d.loss import chamfer_distance 
import torch

N = 100  # Number of random pairs to average over

crystal_liquid_distances = []
liquid_liquid_distances = []
crystal_crystal_distances = []

for _ in range(N):
    points_batch_l1, _ = random.choice(list(dataloader1))
    points_batch_l2, _ = random.choice(list(dataloader1))
    points_batch_c1, _ = random.choice(list(dataloader2))
    points_batch_c2, _ = random.choice(list(dataloader2))
    
    points_l1 = torch.tensor(points_batch_l1[0].numpy()).unsqueeze(0)
    points_l2 = torch.tensor(points_batch_l2[0].numpy()).unsqueeze(0)
    points_c1 = torch.tensor(points_batch_c1[0].numpy()).unsqueeze(0)
    points_c2 = torch.tensor(points_batch_c2[0].numpy()).unsqueeze(0)
    
    crystal_liquid_dist = chamfer_distance(points_l1, points_c1)[0].item()
    liquid_liquid_dist = chamfer_distance(points_l1, points_l2)[0].item()
    crystal_crystal_dist = chamfer_distance(points_c1, points_c2)[0].item()
    
    crystal_liquid_distances.append(crystal_liquid_dist)
    liquid_liquid_distances.append(liquid_liquid_dist)
    crystal_crystal_distances.append(crystal_crystal_dist)

mean_crystal_liquid = sum(crystal_liquid_distances) / N
mean_liquid_liquid = sum(liquid_liquid_distances) / N
mean_crystal_crystal = sum(crystal_crystal_distances) / N

std_crystal_liquid = (sum((x - mean_crystal_liquid) ** 2 for x in crystal_liquid_distances) / N) ** 0.5
std_liquid_liquid = (sum((x - mean_liquid_liquid) ** 2 for x in liquid_liquid_distances) / N) ** 0.5
std_crystal_crystal = (sum((x - mean_crystal_crystal) ** 2 for x in crystal_crystal_distances) / N) ** 0.5

print(f"Mean Chamfer distances over {N} random pairs:")
print(f"Crystal-Liquid: {mean_crystal_liquid:.4f} ± {std_crystal_liquid:.4f}")
print(f"Liquid-Liquid: {mean_liquid_liquid:.4f} ± {std_liquid_liquid:.4f}")
print(f"Crystal-Crystal: {mean_crystal_crystal:.4f} ± {std_crystal_crystal:.4f}")


Mean Chamfer distances over 100 random pairs:
Crystal-Liquid: 0.0885 ± 0.0070
Liquid-Liquid: 0.0835 ± 0.0204
Crystal-Crystal: 0.0831 ± 0.0291


In [28]:
def compare(point_set1, point_set2, k=100, metric='chebyshev'):
    """
    Calculate both PDD and AMD between two point sets.
    """
    def calculate_pdd(points, k):
        dists = np.sort(
            np.linalg.norm(points[:, np.newaxis, :] - points[np.newaxis, :, :], axis=2),
            axis=1
        )
        return dists[:, 1:k+1]

    def calculate_amd(points, k):
        """Calculate the AMD for a single point set."""
        dists = np.sort(
            np.linalg.norm(points[:, np.newaxis, :] - points[np.newaxis, :, :], axis=2),
            axis=1
        )
        return np.mean(dists[:, 1:k+1], axis=1)

    # Calculate PDD
    pdd1 = calculate_pdd(point_set1, k)
    pdd2 = calculate_pdd(point_set2, k)
    dm = cdist(pdd1, pdd2, metric=metric)
    pdd = np.mean(dm)

    # Calculate AMD
    amd1 = calculate_amd(point_set1, k)
    amd2 = calculate_amd(point_set2, k)
    amd = cdist([amd1], [amd2], metric=metric)[0, 0]

    return pdd, amd

def compare_multiple_samples(dataloader1, dataloader2, N=100, k=100, metric='chebyshev'):
    """
    Compare N random pairs of samples and calculate statistics for both PDD and AMD.
    """
    crystal_liquid_distances_pdd = []
    liquid_liquid_distances_pdd = []
    crystal_crystal_distances_pdd = []
    
    crystal_liquid_distances_amd = []
    liquid_liquid_distances_amd = []
    crystal_crystal_distances_amd = []
    
    for _ in range(N):
        points_batch_l1, _ = random.choice(list(dataloader1))
        points_batch_l2, _ = random.choice(list(dataloader1))
        points_batch_c1, _ = random.choice(list(dataloader2))
        points_batch_c2, _ = random.choice(list(dataloader2))
        
        points_l1 = points_batch_l1[0].numpy()
        points_l2 = points_batch_l2[0].numpy()
        points_c1 = points_batch_c1[0].numpy()
        points_c2 = points_batch_c2[0].numpy()
        
        crystal_liquid_pdd, crystal_liquid_amd = compare(points_l1, points_c1, k=k, metric=metric)
        liquid_liquid_pdd, liquid_liquid_amd = compare(points_l1, points_l2, k=k, metric=metric)
        crystal_crystal_pdd, crystal_crystal_amd = compare(points_c1, points_c2, k=k, metric=metric)
        
        crystal_liquid_distances_pdd.append(crystal_liquid_pdd)
        liquid_liquid_distances_pdd.append(liquid_liquid_pdd)
        crystal_crystal_distances_pdd.append(crystal_crystal_pdd)
        
        crystal_liquid_distances_amd.append(crystal_liquid_amd)
        liquid_liquid_distances_amd.append(liquid_liquid_amd)
        crystal_crystal_distances_amd.append(crystal_crystal_amd)
    
    stats = {
        'PDD': {
            'Crystal-Liquid': (np.mean(crystal_liquid_distances_pdd), np.std(crystal_liquid_distances_pdd)),
            'Liquid-Liquid': (np.mean(liquid_liquid_distances_pdd), np.std(liquid_liquid_distances_pdd)),
            'Crystal-Crystal': (np.mean(crystal_crystal_distances_pdd), np.std(crystal_crystal_distances_pdd))
        },
        'AMD': {
            'Crystal-Liquid': (np.mean(crystal_liquid_distances_amd), np.std(crystal_liquid_distances_amd)),
            'Liquid-Liquid': (np.mean(liquid_liquid_distances_amd), np.std(liquid_liquid_distances_amd)),
            'Crystal-Crystal': (np.mean(crystal_crystal_distances_amd), np.std(crystal_crystal_distances_amd))
        }
    }
    
    return stats

# Run the comparison
N = 100
stats = compare_multiple_samples(dataloader1, dataloader2, N=N)

# Print results
print(f"\nResults over {N} random pairs:")
for metric in ['PDD', 'AMD']:
    print(f"\n{metric} distances:")
    for comparison, (mean, std) in stats[metric].items():
        print(f"{comparison}: {mean:.4f} ± {std:.4f}")




Results over 100 random pairs:

PDD distances:
Crystal-Liquid: 0.2877 ± 0.0051
Liquid-Liquid: 0.2739 ± 0.0035
Crystal-Crystal: 0.2995 ± 0.0063

AMD distances:
Crystal-Liquid: 0.0785 ± 0.0211
Liquid-Liquid: 0.0697 ± 0.0272
Crystal-Crystal: 0.0804 ± 0.0346
